### Pre-installs

In [1]:
# %pip install pandas
# %pip install numpy
# %pip install matplotlib
# %pip install seaborn
# %pip install sklearn
# %pip install -U scikit-learn
# %pip install spotipy

### Inputs

In [2]:
input_song = input("Enter your desired song name here!")
input_playlist = input("Enter your desired playlist name here!s")
recc_num = int(input("Enter how many songs you want to be recommended!"))
client_id = input("Enter your Spotify API Client Key here!")
client_secret = input("Enter your Spotify API Secret Key here!")
sp_dc = input("Enter your Spotify sp_dc here!")
sp_key = input("Enter your Spotify sp_key here!")

### CSV & Spotify Information

In [3]:
import pandas as pd
import ast

# Importing the datasets
dataset = pd.read_csv("data/tracks_with_genres_&_language.csv")
df = pd.read_csv("data/tracks.csv")

# Creating the dataframe
df_generated = pd.DataFrame(dataset)
df_generated['genres'] = df_generated['genres'].apply(ast.literal_eval)

In [4]:
# Spotify API Authentication Information
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
# api key
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri="http://localhost/",
                                               scope="playlist-modify-private",
                                               show_dialog=True,
                                               cache_path="token.txt"
                                               ))

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

# # Assuming df_generated is your DataFrame
# # Adjust this part based on your actual DataFrame structure
# # For example, you might need to select specific columns or rows
# # or convert categorical variables to numerical values for visualization

# df_generated.columns
# df_graph = df_generated[['popularity', 'duration_ms', 'explicit',
#        'danceability', 'energy', 'key', 'loudness',
#        'speechiness', 'acousticness', 'instrumentalness', 'liveness',
#        'valence', 'tempo', 'time_signature']]

# sns.set(style="white")  # Set the style of the plot

# # Create a heatmap using seaborn with annotated values for the entire correlation matrix
# plt.figure(figsize=(12, 10))  # Adjust the figure size as needed
# heatmap = sns.heatmap(df_graph.corr(numeric_only=True), annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)

# # Customize the appearance of the heatmap
# heatmap.set_title('Heatmap of df_generated', fontsize=16)

# plt.show()

### Processing the Input Song

In [6]:
# Using Spotify API to search for a song's information based on input and adding the necessary information in a DataFrame.
def search_track(track_name):
    # Search for the track
    results = sp.search(q=track_name, type='track')

    # Check if the track exists
    if results['tracks']['total'] > 0:
        # Get the first track from the results
        track = results['tracks']['items'][0]
        explicit = int(track["explicit"] == True)
        artists = []
        id_artists = []
        for i in range (len(track["artists"])):
            artists.append(track["artists"][i]["name"])
            id_artists.append(track["artists"][i]["id"])

        print(f'Found track: {track["name"]} by {track["artists"][0]["name"]} from the album {track["album"]["name"]}.')
        track_dict = {"id": track["id"], "name": track["name"], "popularity": track["popularity"], 
                      "duration_ms": track["duration_ms"], "explicit": explicit, "artists": str(artists),
                      "id_artists": str(id_artists), "release_date": track["album"]["release_date"]}
        return track_dict
    else:
        print('Track not found')
        return None

# Creating a Single Row DataFrame for the input song.
track_result = search_track(input_song)
td = pd.DataFrame(track_result, index=[0])
td

Found track: SPECIALZ by King Gnu from the album SPECIALZ.


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date
0,0GWNtMohuYUEHVZ40tcnHF,SPECIALZ,86,238986,0,['King Gnu'],['6wxfx1yhyqjCPYwwxJktR2'],2023-09-01


In [7]:
# Obtaining Feature Data from song based on its song_id from previous function and adding them to a DataFrame.
def get_audio_features(track_result):
    song_id = track_result["id"]
    results = sp.audio_features(song_id)

    if results:
        return results[0]
    else:
        print(f'No audio features found for song ID: {song_id}')
        return None

audio_features = get_audio_features(track_result)
af = pd.DataFrame(audio_features, index=[0])
# Taking out the irrevelant features.
af_formatted = af.drop(["type", "id", "uri", "track_href", "analysis_url", "duration_ms"], axis=1)
# Merging both DataFrames to sync up with the dataset's layout.
td = pd.concat([td, af_formatted], axis=1)
td['year'] = td['release_date'].str.extract(r'(\d{4})').astype(int)

In [8]:
td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year
0,0GWNtMohuYUEHVZ40tcnHF,SPECIALZ,86,238986,0,['King Gnu'],['6wxfx1yhyqjCPYwwxJktR2'],2023-09-01,0.566,0.85,...,-4.431,0,0.0599,0.0663,0,0.319,0.722,117.048,4,2023


In [9]:
import ast

# Converting string array into regular array.
td['artists'] = td['artists'].apply(ast.literal_eval)

td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year
0,0GWNtMohuYUEHVZ40tcnHF,SPECIALZ,86,238986,0,[King Gnu],['6wxfx1yhyqjCPYwwxJktR2'],2023-09-01,0.566,0.85,...,-4.431,0,0.0599,0.0663,0,0.319,0.722,117.048,4,2023


In [10]:
# Viewing the input song features.
td['artists_count'] = len(td['artists'][0])
td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count
0,0GWNtMohuYUEHVZ40tcnHF,SPECIALZ,86,238986,0,[King Gnu],['6wxfx1yhyqjCPYwwxJktR2'],2023-09-01,0.566,0.85,...,0,0.0599,0.0663,0,0.319,0.722,117.048,4,2023,1


In [11]:
# Extracting the artists from the input artists array.
expanded_artists = td['artists'].apply(lambda x: pd.Series(x))

# Rename the columns
expanded_artists.columns = [f"artist_{i+1}" for i in range(len(expanded_artists.columns))]

# Concatenate the expanded columns with the original DataFrame
td = pd.concat([td, expanded_artists], axis=1)
td.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count,artist_1
0,0GWNtMohuYUEHVZ40tcnHF,SPECIALZ,86,238986,0,[King Gnu],['6wxfx1yhyqjCPYwwxJktR2'],2023-09-01,0.566,0.85,...,0.0599,0.0663,0,0.319,0.722,117.048,4,2023,1,King Gnu


In [12]:
# Obtaining input artist(s) genre(s) through the Spotify API.
def get_artist_genres(artist_name):
    try:
        results = sp.search(q='artist:' + artist_name, type='artist')
        genres = results['artists']['items'][0]['genres']
        return genres if genres else []
    except IndexError:
        return []
    
for i in range(td['artists_count'][0]):
    td['genres' + str(i+1)] = td['artist_' + str(i+1)].apply(get_artist_genres)
    td['genres' + str(i+1)] = td['genres' + str(i+1)].fillna('[]')
td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count,artist_1,genres1
0,0GWNtMohuYUEHVZ40tcnHF,SPECIALZ,86,238986,0,[King Gnu],['6wxfx1yhyqjCPYwwxJktR2'],2023-09-01,0.566,0.85,...,0.0663,0,0.319,0.722,117.048,4,2023,1,King Gnu,[j-pop]


In [13]:
# Combining all genres into one list.
for i in range(td['artists_count'][0]):
    td['genres'] = td['genres' + str(i+1)] + td['genres' + str(i+1)]
    td.drop(['genres' + str(i+1)], axis=1, inplace=True)
td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count,artist_1,genres
0,0GWNtMohuYUEHVZ40tcnHF,SPECIALZ,86,238986,0,[King Gnu],['6wxfx1yhyqjCPYwwxJktR2'],2023-09-01,0.566,0.85,...,0.0663,0,0.319,0.722,117.048,4,2023,1,King Gnu,"[j-pop, j-pop]"


In [14]:
# Returning only the unique genres for the input.
def get_unique(row):
    return list(set(row))

td['genres'] = td['genres'].apply(get_unique)

In [15]:
# Determining the language from a Lyrics Scraping Library.
import spotify_lyrics_scraper as spotify

token = spotify.getToken(sp_dc, sp_key)
lyrics_data = spotify.getLyrics(token, songName=td['name'][0])
if lyrics_data['status'] == False:
        language = "null"
else:
    # Extract the language from the lyrics data
    language = lyrics_data['message']['lyrics']['language']

td['language'] = language
td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count,artist_1,genres,language
0,0GWNtMohuYUEHVZ40tcnHF,SPECIALZ,86,238986,0,[King Gnu],['6wxfx1yhyqjCPYwwxJktR2'],2023-09-01,0.566,0.85,...,0,0.319,0.722,117.048,4,2023,1,King Gnu,[j-pop],ja


In [ ]:
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

# Many songs do not have lyrics enabled by spotify. So the alternative to that is to use their name as a benchmark for the language.
# Uses language detect library to identify the song name's expected language. Not exactly accurate but it is an acceptable alternative for now until NLP.
def replace_nan_with_language(row):
    if pd.isna(row['language']):  # Check if the value is NaN
        name = row['name']
        if isinstance(name, str) and len(name) > 0:  # Check if name is a non-empty string
            try:
                return detect(name)  # Use detect() output as the value if it's NaN
            except LangDetectException:
                pass  # Handle LangDetectException, e.g., return 'unknown' or None
        # Return None or any other default value if language detection fails or input text is empty
        return None  
    else:
        return row['language']  # Otherwise, keep the original value

td['language'] = td.apply(replace_nan_with_language, axis=1)

td.head()

In [16]:
# Creating a unique identifier.
td['song_name_artist'] = td['name'] + str(td['artists'])

In [17]:
# Defining the values to give priority to later.
language_prority = td['language'][0]
key_priority = td['key'][0]
genre_priority = td['genres'][0]

### Adding Input to Dataset and further processing

In [18]:
# Adding the Input Song to the Dataset DataFrame. Added to the very front.
new_df = pd.concat([td, df_generated], ignore_index=True)
new_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,time_signature,year,artists_count,artist_1,genres,language,song_name_artist,artist_2,artist_3,artist_4
0,0GWNtMohuYUEHVZ40tcnHF,SPECIALZ,86,238986,0,[King Gnu],['6wxfx1yhyqjCPYwwxJktR2'],2023-09-01,0.566,0.850,...,4,2023,1,King Gnu,[j-pop],ja,"SPECIALZ0 [King Gnu]\nName: artists, dtype:...",NaN,NaN,NaN
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,56,160187,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.484,0.265,...,3,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,You'll Never Walk Alone - Mono; 2002 Remaster[...,NaN,NaN,NaN
2,6Pkt6qVikqPBt9bEQy8iTz,A Lover's Concerto,41,159560,0,NaN,['6lH5PpuiMa5SpfjoIOlwCS'],2020-03-13,0.671,0.867,...,4,2020,1,The Toys,[],en,A Lover's Concerto['The Toys'],NaN,NaN,NaN
3,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,40,141987,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.405,0.365,...,4,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,Ferry Cross the Mersey - Mono; 2002 Remaster['...,NaN,NaN,NaN
4,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,34,157093,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.477,0.352,...,4,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,Don't Let the Sun Catch You Crying (Main) - Mo...,NaN,NaN,NaN


In [19]:
new_df['langauge'] = new_df['language'].fillna("null")
new_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,year,artists_count,artist_1,genres,language,song_name_artist,artist_2,artist_3,artist_4,langauge
0,0GWNtMohuYUEHVZ40tcnHF,SPECIALZ,86,238986,0,[King Gnu],['6wxfx1yhyqjCPYwwxJktR2'],2023-09-01,0.566,0.850,...,2023,1,King Gnu,[j-pop],ja,"SPECIALZ0 [King Gnu]\nName: artists, dtype:...",NaN,NaN,NaN,ja
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,56,160187,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.484,0.265,...,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,You'll Never Walk Alone - Mono; 2002 Remaster[...,NaN,NaN,NaN,en
2,6Pkt6qVikqPBt9bEQy8iTz,A Lover's Concerto,41,159560,0,NaN,['6lH5PpuiMa5SpfjoIOlwCS'],2020-03-13,0.671,0.867,...,2020,1,The Toys,[],en,A Lover's Concerto['The Toys'],NaN,NaN,NaN,en
3,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,40,141987,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.405,0.365,...,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,Ferry Cross the Mersey - Mono; 2002 Remaster['...,NaN,NaN,NaN,en
4,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,34,157093,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.477,0.352,...,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,Don't Let the Sun Catch You Crying (Main) - Mo...,NaN,NaN,NaN,en


In [20]:
new_df['song_id_name'] = new_df['id'] + new_df['name']
print(new_df.shape)
new_df.drop_duplicates(subset=["song_id_name"], keep='first', inplace=True)
print(new_df.shape)

(186012, 31)
(186012, 31)


In [21]:
# Counting the number of genres in for each song.
def count_items_in_list(lst):
    return len(lst)

# Apply the function to the DataFrame column
new_df['genres_count'] = new_df['genres'].apply(count_items_in_list)
new_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,artist_1,genres,language,song_name_artist,artist_2,artist_3,artist_4,langauge,song_id_name,genres_count
0,0GWNtMohuYUEHVZ40tcnHF,SPECIALZ,86,238986,0,[King Gnu],['6wxfx1yhyqjCPYwwxJktR2'],2023-09-01,0.566,0.850,...,King Gnu,[j-pop],ja,"SPECIALZ0 [King Gnu]\nName: artists, dtype:...",NaN,NaN,NaN,ja,0GWNtMohuYUEHVZ40tcnHFSPECIALZ,1
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,56,160187,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.484,0.265,...,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,You'll Never Walk Alone - Mono; 2002 Remaster[...,NaN,NaN,NaN,en,6catF1lDhNTjjGa2GxRQNNYou'll Never Walk Alone ...,9
2,6Pkt6qVikqPBt9bEQy8iTz,A Lover's Concerto,41,159560,0,NaN,['6lH5PpuiMa5SpfjoIOlwCS'],2020-03-13,0.671,0.867,...,The Toys,[],en,A Lover's Concerto['The Toys'],NaN,NaN,NaN,en,6Pkt6qVikqPBt9bEQy8iTzA Lover's Concerto,0
3,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,40,141987,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.405,0.365,...,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,Ferry Cross the Mersey - Mono; 2002 Remaster['...,NaN,NaN,NaN,en,4aSw1QJIMwYSoDEgzgdCJLFerry Cross the Mersey -...,9
4,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,34,157093,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.477,0.352,...,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,Don't Let the Sun Catch You Crying (Main) - Mo...,NaN,NaN,NaN,en,0ZMMtH875IR2TfkyC4PolDDon't Let the Sun Catch ...,9


In [22]:
print(new_df.shape)
new_df = new_df[(new_df['genres_count'] != 0)]
print(new_df.shape)

(186012, 32)
(166252, 32)


### Supersetting genres

In [23]:
# Determining the frequency of the genre. Giving the input song's genre priority to go first.
genre_extract = new_df['genres'].reset_index()
genre_wo_index = genre_extract['genres']
tally = genre_wo_index.count()
genre_dict = {}

for i in range (tally):
    input = genre_wo_index.loc[i]
    if len(input) > 0:
        for n in input:
            if n in genre_dict.keys():
                genre_dict[n] += 1
            else:
                genre_dict[n] = 1

print(genre_dict)

{'j-pop': 4599, 'adult standards': 1216, 'bubblegum pop': 416, 'merseybeat': 529, 'rock-and-roll': 650, 'british invasion': 495, 'classic uk pop': 704, 'brill building pop': 763, 'rockabilly': 550, 'folk rock': 933, 'easy listening': 126, 'lounge': 283, 'british blues': 265, 'canadian blues': 54, 'singer-songwriter': 312, 'lilith': 307, 'canadian singer-songwriter': 107, 'folk': 393, 'rock': 3193, 'mellow gold': 1141, 'jazz': 304, 'swing': 56, 'big band': 94, 'jazz piano': 73, 'stride': 75, 'vocal jazz': 355, 'cool jazz': 274, 'harlem renaissance': 23, 'jazz saxophone': 71, 'contemporary jazz': 73, 'jazz quartet': 20, 'jazz fusion': 297, 'swedish jazz': 134, 'free jazz': 58, 'avant-garde jazz': 49, 'contemporary post-bop': 57, 'italian jazz': 15, 'soundtrack': 456, 'italian soundtrack': 28, 'vintage italian soundtrack': 34, 'classic soundtrack': 89, 'jazz trumpet': 176, 'hard bop': 204, 'roots rock': 250, 'psychedelic rock': 592, 'classic rock': 1358, 'acid rock': 67, 'electric blues':

In [25]:
genres_df = pd.Series(genre_dict)
genres_df.head()

j-pop              4599
adult standards    1216
bubblegum pop       416
merseybeat          529
rock-and-roll       650
dtype: int64

In [26]:
# Summary Statistics of genres.
genres_df.T.describe()

count    4229.000000
mean      130.132419
std       393.312889
min         1.000000
25%         4.000000
50%        19.000000
75%        85.000000
max      6880.000000
dtype: float64

In [27]:
# genres_df.to_csv("data/genre_frequency.csv")

In [28]:
# Creating the genre supersets, with priority to the input song's genre.
# Since the input song is the first one to be run, its genre will be able to start the classification of other sub-genres under it.

# This can be further improved. Allowing for later genres to take over earlier genres as the superset if it more general.
# E.g.: Detroit Hip Hop is found before general hip hop. Currently they will be seperated into 2 seperate genres because the first word 'detroit' is
# not 'hip hop' and is thus overlooked.

def generate_superset_mapping(genre_dict):
    superset_mapping = {}

    for genre, frequency in genre_dict.items():
        subset_added = False
        for superset in list(superset_mapping.keys()):
            if genre in superset:
                superset_mapping[genre] = [genre]
                superset_mapping[genre].extend(superset_mapping[superset])
                del superset_mapping[superset]
                subset_added = True
                break
            elif superset in genre:
                if superset in superset_mapping:
                    superset_mapping[superset].append(genre)
                    subset_added = True
                    break
        if not subset_added:
            superset_mapping[genre] = [genre]

    return superset_mapping

# Generate superset mapping
superset_mapping = generate_superset_mapping(genre_dict)

# Output the superset mapping
for superset, subsets in superset_mapping.items():
    print(f"{superset}: {subsets}")


adult standards: ['adult standards', 'deep adult standards']
bubblegum pop: ['bubblegum pop']
merseybeat: ['merseybeat']
british invasion: ['british invasion']
brill building pop: ['brill building pop']
rockabilly: ['rockabilly', 'uk rockabilly', 'finnish rockabilly', 'german rockabilly', 'neo-rockabilly', 'swedish rockabilly', 'spanish rockabilly', 'rockabilly en espanol', 'japanese rockabilly', 'canadian rockabilly']
easy listening: ['easy listening']
lounge: ['lounge', 'lounge house', 'chill lounge', 'italian lounge', 'sunset lounge']
canadian blues: ['canadian blues']
singer-songwriter: ['singer-songwriter', 'canadian singer-songwriter', 'nashville singer-songwriter', 'taiwan singer-songwriter', 'singaporean singer-songwriter', 'indian singer-songwriter', 'scottish singer-songwriter', 'japanese singer-songwriter', 'swedish singer-songwriter', 'british singer-songwriter', 'turkish singer-songwriter', 'irish singer-songwriter', 'norwegian singer-songwriter', 'nz singer-songwriter', '

In [29]:
# Supersetting the weighted genres for later.

superset_genre_priority = []
if len(genre_priority) != 0:
    for i in genre_priority:
        for n in superset_mapping:
            if i in n:
                superset_genre_priority.append(n)
print(superset_genre_priority)

[]


In [30]:
# Now with the superset mapped out, apply it onto the all the songs to determine their supersetted genre(s).
# This can be improved. Instead of only returning the first superset genre, it should return multiple in an array.

import pandas as pd

new_df['supersetted_genres'] = new_df['genres'].apply(lambda genres: [m for n in genres for m in superset_mapping if n in m])

In [31]:
new_df['supersetted_genres'].value_counts().count()

6117

In [32]:
new_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,genres,language,song_name_artist,artist_2,artist_3,artist_4,langauge,song_id_name,genres_count,supersetted_genres
0,0GWNtMohuYUEHVZ40tcnHF,SPECIALZ,86,238986,0,[King Gnu],['6wxfx1yhyqjCPYwwxJktR2'],2023-09-01,0.566,0.850,...,[j-pop],ja,"SPECIALZ0 [King Gnu]\nName: artists, dtype:...",NaN,NaN,NaN,ja,0GWNtMohuYUEHVZ40tcnHFSPECIALZ,1,[]
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,56,160187,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.484,0.265,...,"[adult standards, bubblegum pop, merseybeat, r...",en,You'll Never Walk Alone - Mono; 2002 Remaster[...,NaN,NaN,NaN,en,6catF1lDhNTjjGa2GxRQNNYou'll Never Walk Alone ...,9,"[adult standards, bubblegum pop, merseybeat, b..."
3,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,40,141987,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.405,0.365,...,"[adult standards, bubblegum pop, merseybeat, r...",en,Ferry Cross the Mersey - Mono; 2002 Remaster['...,NaN,NaN,NaN,en,4aSw1QJIMwYSoDEgzgdCJLFerry Cross the Mersey -...,9,"[adult standards, bubblegum pop, merseybeat, b..."
4,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,34,157093,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.477,0.352,...,"[adult standards, bubblegum pop, merseybeat, r...",en,Don't Let the Sun Catch You Crying (Main) - Mo...,NaN,NaN,NaN,en,0ZMMtH875IR2TfkyC4PolDDon't Let the Sun Catch ...,9,"[adult standards, bubblegum pop, merseybeat, b..."
5,1hx7X9cMXHWJjknb9O6Ava,The September Of My Years - Live At The Sands ...,26,187333,0,NaN,['1Mxqyy3pSjf8kZZL4QVxS0'],2018-05-04,0.319,0.201,...,"[easy listening, adult standards, lounge]",en,The September Of My Years - Live At The Sands ...,NaN,NaN,NaN,en,1hx7X9cMXHWJjknb9O6AvaThe September Of My Year...,3,"[easy listening, adult standards, lounge]"


### Encoding

In [33]:
# One-hot Encoding of all the superset genres of each song in the dataframe.

genres_dummies = pd.get_dummies(new_df['supersetted_genres'].apply(pd.Series).stack()).groupby(level=0).sum()

# Concatenate the binary columns with the original DataFrame
df_2 = pd.concat([new_df, genres_dummies], axis=1)

In [34]:
df_2.iloc[:,0:20].describe()

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
count,166252.000000,1.662520e+05,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000
mean,40.123259,2.328365e+05,0.107283,0.606616,0.653452,5.330065,-7.320525,0.609117,0.091713,0.299999,0.063974,0.207368,0.535138,121.775839,3.929324
std,17.117333,1.013132e+05,0.309474,0.157524,0.216638,3.565031,3.642158,0.487950,0.112539,0.292922,0.206059,0.183005,0.249318,29.153030,0.361895
min,0.000000,4.937000e+03,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,1.911600e+05,0.000000,0.504000,0.511000,2.000000,-8.686000,0.000000,0.034200,0.039200,0.000000,0.096900,0.335000,98.006000,4.000000
50%,41.000000,2.237130e+05,0.000000,0.617000,0.681000,5.000000,-6.603000,1.000000,0.047800,0.199000,0.000002,0.131000,0.536000,121.927000,4.000000
75%,52.000000,2.624270e+05,0.000000,0.721000,0.829000,9.000000,-5.028000,1.000000,0.092800,0.514000,0.000489,0.265000,0.741000,139.952000,4.000000
max,100.000000,4.995083e+06,1.000000,0.991000,1.000000,11.000000,2.854000,1.000000,0.966000,0.996000,1.000000,1.000000,1.000000,220.099000,5.000000


In [35]:
# Determining how many keys there are.
df_2.groupby('key').count()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,zen,zespol dzieciecy,zhongguo feng,zikir,zillertal,zim urban groove,zolo,zouglou,zouk,zurich indie
key,,,,,,,,,,,,,,,,,,,,,
0,19586,19586,19586,19586,19586,0,19586,19586,19586,19586,...,11443,11443,11443,11443,11443,11443,11443,11443,11443,11443
1,14502,14502,14502,14502,14502,0,14502,14502,14502,14502,...,7747,7747,7747,7747,7747,7747,7747,7747,7747,7747
2,17337,17337,17337,17337,17337,0,17337,17337,17337,17337,...,9877,9877,9877,9877,9877,9877,9877,9877,9877,9877
3,5024,5024,5024,5024,5024,0,5024,5024,5024,5024,...,2875,2875,2875,2875,2875,2875,2875,2875,2875,2875
4,13626,13626,13626,13626,13626,0,13626,13626,13626,13626,...,7608,7608,7608,7608,7608,7608,7608,7608,7608,7608
5,13199,13199,13199,13199,13199,0,13199,13199,13199,13199,...,7733,7733,7733,7733,7733,7733,7733,7733,7733,7733
6,11253,11253,11253,11253,11253,1,11253,11253,11253,11253,...,6154,6154,6154,6154,6154,6154,6154,6154,6154,6154
7,19669,19669,19669,19669,19669,0,19669,19669,19669,19669,...,11364,11364,11364,11364,11364,11364,11364,11364,11364,11364
8,10079,10079,10079,10079,10079,0,10079,10079,10079,10079,...,5556,5556,5556,5556,5556,5556,5556,5556,5556,5556


In [36]:
df_2['key'].max()

11

In [37]:
# One-hot encoding the keys.
key_encoded = pd.get_dummies(df_2['key'], columns=[i for i in range(12)], prefix='key').astype(int)

In [38]:
key_encoded.head()

,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,0,0,0,0,0,0,1,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,1,0,0,0,0


In [39]:
# Adding keys encoding into the dataset.
df_2 = pd.concat([df_2, key_encoded], axis=1)

In [40]:
print(df_2.shape)
df_2.drop_duplicates(subset=["song_id_name"], keep='first', inplace=True)
print(df_2.shape)

(166252, 1514)
(166252, 1514)


In [41]:
# Checking all the languages.
df_2['langauge'].value_counts().keys()

Index(['en', 'es', 'de', 'pt', 'id', 'it', 'tl', 'tr', 'ko', 'fr', 'fi', 'ja',
       'ca', 'nl', 'sw', 'th', 'ro', 'no', 'et', 'so', 'hu', 'he', 'ru', 'pl',
       'sv', 'da', 'af', 'lt', 'hr', 'sl', 'cy', 'sk', 'zh-cn', 'cs', 'lv',
       'bg', 'zh-tw', 'vi', 'null', 'mk', 'sq', 'uk', 'iw', 'ar', 'el', 'z1',
       'hi', 'zu', 'xh', 'tt', 'ms', 'fa', 'ta', 'mi', 'st', 'sn', 'sm', 'gn',
       'rn', 'qu', 'nn', 'eu', 'tn', 'az', 'am', 'ht', 'ln', 'la', 'ia', 'sr',
       'ne', 'is', 'br', 'co', 'm7', 'ug', 'gl', 'ur'],
      dtype='object', name='langauge')

In [42]:
df_2['language'].head()

,language,language
0,ja,NaN
1,en,0.0
3,en,0.0
4,en,0.0
5,en,0.0


In [43]:
# Encoding all the languages.
df_2['language'] = df_2['language'].fillna('null')
language_encoded = pd.get_dummies(df_2['language'], prefix='lang').astype(int)
language_encoded.head()

,lang_af,lang_am,lang_ar,lang_az,lang_bg,lang_br,lang_ca,lang_co,lang_cs,lang_cy,...,lang_ur,lang_vi,lang_xh,lang_z1,lang_zh-cn,lang_zh-tw,lang_zu,lang_0.0,lang_1.0,lang_null
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [44]:
language_encoded = language_encoded.drop(['lang_0.0', 'lang_1.0', 'lang_null'], axis=1)
language_encoded.head()

,lang_af,lang_am,lang_ar,lang_az,lang_bg,lang_br,lang_ca,lang_co,lang_cs,lang_cy,...,lang_ug,lang_uk,lang_ur,lang_vi,lang_xh,lang_z1,lang_zh-cn,lang_zh-tw,lang_zu,lang_null
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
# Adding language encoding into the dataset.
df_2 = pd.concat([df_2, language_encoded], axis=1)

In [46]:
print(df_2.shape)
df_2.drop_duplicates(subset=["song_id_name"], keep='first', inplace=True)
print(df_2.shape)

(166252, 1593)
(166252, 1593)


In [47]:
# Removing unnecessary features.
df_2 = df_2.drop(['id_artists', 'release_date', 'popularity', 'key', 'danceability', 'liveness', 'artists_count', 'song_name_artist', 'artist_1', 'artist_2', 'artist_3', 'artist_4', 'genres', 'supersetted_genres', 'explicit', 'time_signature', 'language'], axis=1)
df_2.head()

,id,name,duration_ms,artists,energy,loudness,mode,speechiness,acousticness,instrumentalness,...,lang_ug,lang_uk,lang_ur,lang_vi,lang_xh,lang_z1,lang_zh-cn,lang_zh-tw,lang_zu,lang_null
0,0GWNtMohuYUEHVZ40tcnHF,SPECIALZ,238986,[King Gnu],0.850,-4.431,0,0.0599,0.0663,0.000000,...,0,0,0,0,0,0,0,0,0,1
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,160187,NaN,0.265,-11.101,1,0.0322,0.3940,0.000000,...,0,0,0,0,0,0,0,0,0,0
3,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,141987,NaN,0.365,-10.226,0,0.0289,0.2550,0.000005,...,0,0,0,0,0,0,0,0,0,0
4,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,157093,NaN,0.352,-14.165,1,0.0300,0.4060,0.000000,...,0,0,0,0,0,0,0,0,0,0
5,1hx7X9cMXHWJjknb9O6Ava,The September Of My Years - Live At The Sands ...,187333,NaN,0.201,-17.796,1,0.0623,0.8870,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [48]:
# Filling in any null values. (In case)

df_2 = df_2.fillna(0)
df_2.head()

,id,name,duration_ms,artists,energy,loudness,mode,speechiness,acousticness,instrumentalness,...,lang_ug,lang_uk,lang_ur,lang_vi,lang_xh,lang_z1,lang_zh-cn,lang_zh-tw,lang_zu,lang_null
0,0GWNtMohuYUEHVZ40tcnHF,SPECIALZ,238986,[King Gnu],0.850,-4.431,0,0.0599,0.0663,0.000000,...,0,0,0,0,0,0,0,0,0,1
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,160187,0,0.265,-11.101,1,0.0322,0.3940,0.000000,...,0,0,0,0,0,0,0,0,0,0
3,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,141987,0,0.365,-10.226,0,0.0289,0.2550,0.000005,...,0,0,0,0,0,0,0,0,0,0
4,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,157093,0,0.352,-14.165,1,0.0300,0.4060,0.000000,...,0,0,0,0,0,0,0,0,0,0
5,1hx7X9cMXHWJjknb9O6Ava,The September Of My Years - Live At The Sands ...,187333,0,0.201,-17.796,1,0.0623,0.8870,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [49]:
df_2.shape

(166252, 1575)

### Weighting

In [75]:
# Applying weight to input language.
df_2['lang_' + language_prority] = df_2['lang_' + language_prority] * 10
df_2[['name','lang_' + language_prority]].head()

,name,lang_ja
0,SPECIALZ,10.0
1,You'll Never Walk Alone - Mono; 2002 Remaster,0.0
3,Ferry Cross the Mersey - Mono; 2002 Remaster,0.0
4,Don't Let the Sun Catch You Crying (Main) - Mono,0.0
5,The September Of My Years - Live At The Sands ...,0.0


In [74]:
# Reducing weight from non-input language.
for i in df_2.columns:
    if 'lang_' in i:
        if i != ('lang_' + language_prority):
            df_2[i] = df_2[i] / 10
df_2.head()

0    0.0
1    0.1
3    0.1
4    0.1
5    0.1
Name: lang_en, dtype: float64

In [57]:
# Applying weight to input key.
df_2['key_' + str(key_priority)] = df_2['key_' + str(key_priority)] * 10
df_2[['name','key_' + str(key_priority)]].head()

,name,key_6
0,SPECIALZ,10
1,You'll Never Walk Alone - Mono; 2002 Remaster,0
3,Ferry Cross the Mersey - Mono; 2002 Remaster,10
4,Don't Let the Sun Catch You Crying (Main) - Mono,0
5,The September Of My Years - Live At The Sands ...,0


In [76]:
# Reducing weight from non-input key.
for i in df_2.columns:
    if 'key_' in i:
        if i != ('key_' + str(key_priority)):
            df_2[i] = df_2[i] / 10
df_2['key_7'].head()

0    0.0
1    0.0
3    0.0
4    0.0
5    0.1
Name: key_7, dtype: float64

In [ ]:
# Applying weight to input genre(s).
superset_genre_priority = list(set(superset_genre_priority))

if len(superset_genre_priority) != 0:
    for i in superset_genre_priority:
        df_2[i] = df_2[i] * 10
df_2[superset_genre_priority].head()

""
1
3
4
5
6


In [82]:
# Reducing weight from non-input genre(s).
for n in superset_genre_priority:
    for i in df_2.columns:
        if i in superset_mapping.keys():
            if i != (n):
                df_2[i] = df_2[i] / 10
df_2['pop'].head()

0    0.0
1    0.0
3    0.0
4    0.0
5    0.0
Name: hip hop, dtype: float64

### Cosine Similarity & Output Extraction

In [83]:
import numpy as np
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

# Feature selection, removing non-useful rows.
def processing(df_2):
    # Normalize numerical features
    numerical_features = ['loudness', 'tempo']
    scaler = MinMaxScaler()
    df_2[numerical_features] = scaler.fit_transform(df_2[numerical_features])
    
    # Standardize Year
    df_2['standardized_year'] = scaler.fit_transform(df_2[['year']])
    df_2 = df_2.drop(['year'], axis=1)

    # Create a feature matrix
    feature_matrix = df_2.drop(['id', 'name', 'artists', 'song_id_name', 'langauge'], axis=1)

    # Apply normalization to the entire feature matrix
    feature_matrix = scaler.fit_transform(feature_matrix)

    # This is the input song. We are isolating it from the dataframe first.
    input_song = feature_matrix[0]
    
    arr = []

    # Returns comparison value of first song in dataset to all others in an array.
    # DataFrame can be customized before Feature Selection to change which song the subject of comparison should be.
    for i in range(1, len(feature_matrix)):
        # Take the i-th song from the feature matrix
        compare_song = feature_matrix[i]
        # Perform cosine similarity.
        cosine_sim = cosine_similarity([input_song], [compare_song])
        cs_value = cosine_sim[0][0]  # Take the single similarity value between the input song and the i-th song.
        arr.append(cs_value)

    return arr

arr = processing(df_2)
arr

[0.23627817193905243,
 0.42689433605416777,
 0.25536264541897136,
 0.2864143295040233,
 0.27754805687261236,
 0.3416161818984902,
 0.45750800828796123,
 0.2554657428015188,
 0.41991469152525424,
 0.32424342682366947,
 0.2562522894420272,
 0.2580814908043731,
 0.3264200002670997,
 0.2930964286279427,
 0.2844427841985346,
 0.2673328604640586,
 0.43092448805116146,
 0.42332150816368685,
 0.3988558571956776,
 0.27754834152639807,
 0.4045073692595066,
 0.3364781392403121,
 0.333157933473779,
 0.27022667756486557,
 0.3004067651334659,
 0.30970070873625444,
 0.2545338470941526,
 0.1719258599118676,
 0.3156986093386693,
 0.17830297916157645,
 0.5162900121783098,
 0.39053690195011,
 0.34463544972278126,
 0.3300689359744573,
 0.30162380710478354,
 0.41514101242180607,
 0.362130045174764,
 0.4286692440770424,
 0.30505644254546427,
 0.3472454142304445,
 0.28867921902689,
 0.27604003037875025,
 0.4117270093544488,
 0.3976261542624854,
 0.3693297893285322,
 0.3816858412804195,
 0.3184555293287199,
 

In [84]:
max(arr[1:])

0.9959162226597108

In [85]:
def compile_suggestions(arr):
    suggestions = []
    
    # Compiling all song ids that are above the pre-determined threshold for 'like the first song'.
    for i in range(len(arr)):
        if arr[i] > 0.9:
            suggestions.append(i)
    
    print(suggestions)
    return suggestions
suggestions = compile_suggestions(arr)

[7861, 9399, 9904, 11177, 14290, 23345, 23990, 32294, 33496, 36233, 58374, 61831, 151720, 151845, 151969, 152713, 152728, 152742, 153016, 153088, 153255, 153377, 153431, 153518, 153875, 154092, 154529, 154634, 154670, 154815, 154972, 155001, 155210, 155213, 155276, 155306, 155347, 155498, 155632, 155646, 155792, 155822, 155957, 156009, 156119, 156123, 156223, 156300, 156304, 156582, 156772, 156802, 156889, 156948, 156970, 157010, 157059, 157064, 157268, 157314, 157320, 157391, 157469, 157493, 157495, 157584, 157593, 157693, 157713, 157722, 157822, 157880, 157929, 157976, 158021, 158023, 158114, 158194, 158195, 158251, 158326, 158393, 158443, 158481, 158503, 158509, 158533, 158602, 158638, 158662, 158683]


In [86]:
len(suggestions)

91

In [87]:
# Extracting out the end result information.
df_main = df_2[df_2.index.isin(suggestions)]

df_main.head()

,id,name,duration_ms,artists,energy,loudness,mode,speechiness,acousticness,instrumentalness,...,lang_uk,lang_ur,lang_vi,lang_xh,lang_z1,lang_zh-cn,lang_zh-tw,lang_zu,lang_null,standardized_year
7861,7u5B4ptccyg8T7NPfByhE7,Tere Naina,255721,0,0.377,0.791931,0,0.0511,0.544,0.000077,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.347826
11177,78FclSSqu1fMOMPsEBeOsI,Free Hand - 2012 Remaster,372880,0,0.578,0.825834,1,0.0369,0.285,0.023000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.521739
14290,7osHjFGYUmIMB6eAiU9HDs,Querida,295280,0,0.586,0.869873,1,0.0286,0.321,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.652174
23345,4bzS0aUqNJIKiXk86k4tol,Soke,218867,0,0.497,0.776816,1,0.1040,0.599,0.000006,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.652174
32294,2yDaWqECegHdiFOuez5gdO,Sunset (feat. Shaggy & Nicky Jam),238080,0,0.882,0.908614,1,0.0389,0.118,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.652174


In [88]:
len(df_main) # There is one less for the last 3 features as the input song did not go through the extracted year's steps.
# df_final['standardized_year'].describe()

80

In [89]:
df_main = df_main.iloc[1:]

df_main = df_main.drop('artists', axis=1)

id_artist_mapping = df.set_index('id')['artists'].to_dict()

# Add 'artist' column to df1 using the mapping
df_main['artists'] = df_main['id'].map(id_artist_mapping)
df_main.head()

,id,name,duration_ms,energy,loudness,mode,speechiness,acousticness,instrumentalness,valence,...,lang_ur,lang_vi,lang_xh,lang_z1,lang_zh-cn,lang_zh-tw,lang_zu,lang_null,standardized_year,artists
11177,78FclSSqu1fMOMPsEBeOsI,Free Hand - 2012 Remaster,372880,0.578,0.825834,1,0.0369,0.285,0.023000,0.662,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.521739,['Gentle Giant']
14290,7osHjFGYUmIMB6eAiU9HDs,Querida,295280,0.586,0.869873,1,0.0286,0.321,0.000000,0.509,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.652174,"['Juan Gabriel', 'Juanes']"
23345,4bzS0aUqNJIKiXk86k4tol,Soke,218867,0.497,0.776816,1,0.1040,0.599,0.000006,0.883,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.652174,['Burna Boy']
32294,2yDaWqECegHdiFOuez5gdO,Sunset (feat. Shaggy & Nicky Jam),238080,0.882,0.908614,1,0.0389,0.118,0.000000,0.625,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.652174,"['Farruko', 'Shaggy', 'Nicky Jam']"
33496,1UHcnP31sAKOJFrEnyRNUz,Amantes y Amigos,182627,0.712,0.889522,1,0.1530,0.369,0.000000,0.468,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.869565,"['Arcangel', 'Sech']"


### Recommendations Output

In [91]:
# Generating recc_num of the matched songs at random.
def recommend(df_main, recc_num):
    recs = []
    if len(df_main) == 0:
        print("Sorry! There are no songs similar enough to " + input_song + "!")
    elif len(df_main) < recc_num:
        recc_num = len(df_main)
        print("Uh oh! We only found " + str(recc_num) + " recommendations!")
    print('Here are your recommendations!')
    for c in range(recc_num):
        recc = df_main.sample().to_dict()
        name = list(recc['name'].values())[0]
        artists = (list(recc['artists'].values())[0])[1:len(list(recc['artists'].values())[0])-1]
        year = str(list(recc['year'].values())[0])
        print(str(c + 1) + '. ' + name + ' by ' + artists + ' published in ' + year)
        df_main = df_main.drop(list(recc['id'])[0])
        recs.append((name, year))
    return recs
# May want to try a filter for repreventing duplicate songs in the list.
recs_for_user = recommend(df_main, recc_num) # End Product

Here are your recommendations!
1. 口笛 by 'Mr.Children' published in 2000
2. Ott Leszek Melletted by 'L.L. Junior' published in 2004
3. Lazy Generation - Edited by 'The F-Ups' published in 2004
4. Valaki Beszorult Egy Zetorba by 'Rózsaszín Pittbull' published in 2002
5. Petőfi Sándor by 'Nemazalány', 'LIL G' published in 2018
6. Touchstone by 'Solarstone' published in 2010
7. Conni zieht um - Teil 01 by 'Conni' published in 2003
8. Érj hozzám by 'Rúzsa Magdolna' published in 2016
9. Dani Su Bez Broja by 'Magazin', 'Esma Redzepova' published in 2002
10. Takard el by 'AWS' published in 2011


C:\Users\Kwang Zhe\AppData\Local\Temp\ipykernel_23200\2734259922.py:11: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  recc = df_main.sample().to_dict()
C:\Users\Kwang Zhe\AppData\Local\Temp\ipykernel_23200\2734259922.py:11: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  recc = df_main.sample().to_dict()
C:\Users\Kwang Zhe\AppData\Local\Temp\ipykernel_23200\2734259922.py:11: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  recc = df_main.sample().to_dict()
C:\Users\Kwang Zhe\AppData\Local\Temp\ipykernel_23200\2734259922.py:11: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  recc = df_main.sample().to_dict()
C:\Users\Kwang Zhe\AppData\Local\Temp\ipykernel_23200\2734259922.py:11: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  recc = df_main.sample().to_dict()
C:\Users\Kwang Zhe\AppData\Local\Temp\ipykernel_23200\2734259922.py:11: Use

In [92]:
user_id = sp.current_user()["id"]
song_uris = []
for song in recs_for_user:
    result = sp.search(q=f"track:{song[0]} year:{format(float(song[1]), '.0f')}", type="track")
    print(result)
    try:
        uri = result["tracks"]["items"][0]["uri"]
        song_uris.append(uri)
    except IndexError:
        print(f"{song[0]} doesn't exist in Spotify. Skipped.")

playlist = sp.user_playlist_create(user=user_id, name=input_playlist, public=False)
sp.playlist_add_items(playlist_id=playlist["id"], items=song_uris)

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3A%E5%8F%A3%E7%AC%9B+year%3A2000&type=track&offset=0&limit=10', 'items': [{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1qma7XhwZotCAucL7NHVLY'}, 'href': 'https://api.spotify.com/v1/artists/1qma7XhwZotCAucL7NHVLY', 'id': '1qma7XhwZotCAucL7NHVLY', 'name': 'Mr.Children', 'type': 'artist', 'uri': 'spotify:artist:1qma7XhwZotCAucL7NHVLY'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 

{'snapshot_id': 'Miw1N2I1N2Y2ZTFmNWE0MTVkMjg0YWE2ZDU3MDkxMmM5NzQzYzQ4OWZh'}